In [2]:
import os
import csv
import time

import sys
sys.path.insert(0, '../scripts/')

from graphreader import read_graph

from greedy import greedy_mwds

In [3]:
dir_instances = sorted(os.listdir("../instances"))

In [10]:
import pulp


def ilpmodel(vertices, edges):
    model = pulp.LpProblem("Minimum Weighted Directed Domination Set", pulp.LpMinimize)

    x = {}
    # definisemo varijable za cvorove u grafu
    for i in vertices.items():
        x[i[0]] = pulp.LpVariable("x_" + str(i[0]), 0, 1, pulp.LpBinary)

    # definisemo tezine cvorova
    weights = {}
    for i in vertices.items():
        weights[i[0]] = i[1]

    # dodajemo funkciju cilja
    model += sum(weights[u] * x[u] for u in x)

    # dodajemo ogranicenja
    for u, v in edges:
      model += x[u] + x[v] >= 1

    # pokrecemo model
    model.solve()

    # kreiramo dominirajuci skup
    domination_set = []
    for u in x:
      if x[u].value() == 1:
        domination_set.append(u)

    objective_value = pulp.value(model.objective)

    # vracamo dominirajuci skup i tezinu
    return domination_set, objective_value


In [ ]:
with open('../results/results.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # header
    writer.writerow(['Instances', 'Greedy algorithm', 'Variable neighborhood search (VNS)', 'PuLP linar programming'])

    # prolazak kroz razlicite dimenzije problema
    for subdir in dir_instances:
        dim_path = os.path.join("../instances/", subdir)
        print("Dimension - ", subdir)

        if subdir.endswith("0"):
            instance_dim = 0
            num_vertices = 5
        elif subdir.endswith("1"):
            instance_dim = 1
            num_vertices = 500
        else:
            instance_dim = 2
            num_vertices = 1000

        # prolazak kroz sve instance odredjene dimenzije
        for instance in sorted(os.listdir(os.path.abspath(dim_path))):

            instance_path = os.path.join(dim_path, instance)
            vertices, edges = read_graph(instance_path, num_vertices)
            vertices_list = vertices
            num_edges = len(edges)
            vertices = dict(vertices)

            start = time.time()
            set, weight = greedy_mwds(vertices, edges)
            end = time.time()

            elapsed = end - start
            greedy_results = "|V| = " + str(len(set)) + "\n" + "weight = " + str(weight) +"\n" + "T(s) = " + f"{elapsed:.3f}"


            start = time.time()
            set, weight = ilpmodel(vertices, edges)
            end = time.time()

            elapsed = end - start
            ilp_results = "|V| = " + str(len(set)) + "\n" + "weight = " + str(weight) +"\n" + "T(s) = " + f"{elapsed:.3f}"


            instance_info = "Instance " + str(instance_dim) + "lvl\n" + str("|V| = ") + str(num_vertices) + "\n |E| = " + str(num_edges)
            writer.writerow([instance_info, greedy_results, 0, ilp_results])



Dimension -  level0
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/vladimir/anaconda3/envs/operaciona_istrazivanja/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/a86bcf1a86e5412ba3f5a0477ba6f77a-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/a86bcf1a86e5412ba3f5a0477ba6f77a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 39 RHS
At line 46 BOUNDS
At line 52 ENDATA
Problem MODEL has 6 rows, 5 columns and 12 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 156 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 3 rows, 5 columns (5 integer (5 of which binary)) and 